In [1]:
dts <- read.csv('hg19.cage_peak_phase1and2combined_tpm_ann_decoded.osc.txt.gz.extract.tsv',sep = '\t' ,header=TRUE)

In [2]:
dts <- dts[,-1:-2]

In [3]:
dts <- na.omit(dts)
dts <- subset(dts, uniprot_id!="")

In [4]:
library(plyr)
small_dts <- ddply(dts, 'uniprot_id', numcolwise(sum))

In [5]:
rownames(small_dts) <- small_dts$uniprot_id
small_dts <- small_dts[,-1]
tail(small_dts)

,Astrocyte...cerebellum..donor1.CNhs11321.11500.119F6,Astrocyte...cerebral.cortex..donor1.CNhs10864.11235.116D2,brain..adult..donor1.CNhs11796.10084.102B3,brain..adult..pool1.CNhs10617.10012.101C3,brain..fetal..pool1.CNhs11797.10085.102B4,breast..adult..donor1.CNhs11792.10080.102A8,cerebellum...adult..donor10196.CNhs13799.10173.103C2,cerebellum..adult..donor10252.CNhs12323.10166.103B4,cerebellum..newborn..donor10223.CNhs14075.10357.105E6,dura.mater..adult..donor1.CNhs10648.10041.101F5,⋯,thalamus...adult..donor10196.CNhs13794.10168.103B6,thalamus..adult..donor10252.CNhs12314.10154.103A1,thalamus..adult..donor10258..tech_rep1.CNhs14223.10370.105G1,thalamus..adult..donor10258..tech_rep2.CNhs14551.10370.105G1,thalamus..newborn..donor10223.CNhs14084.10366.105F6,throat..fetal..donor1.CNhs11770.10061.101H7,thyroid..fetal..donor1.CNhs11769.10060.101H6,tongue.epidermis..fungiform.papillae...donor1.CNhs13460.10288.104F9,umbilical.cord..fetal..donor1.CNhs11765.10057.101H3,uterus..fetal..donor1.CNhs11763.10055.101H1
"uniprot:Q9Y6Y8,uniprot:F5H0L8",29.55,34.63,35.87,26.53,31.94,42.77,64.08,34.17,42.93,43.73,⋯,53.07,24.72,31.06,41.81,43.47,51.66,62.45,51.60,44.58,39.06
"uniprot:Q9Y6Y9,uniprot:D0VAW8",0.00,0.00,0.00,0.37,0.00,1.44,0.00,0.22,0.52,2.97,⋯,1.66,1.04,0.00,0.00,2.14,0.24,0.61,0.00,0.00,0.81
"uniprot:Q9Y6Y9,uniprot:E5RJJ7",0.49,0.29,2.91,13.08,2.00,18.12,1.53,3.10,4.66,29.19,⋯,11.61,12.10,7.06,7.25,14.96,5.24,1.41,2.58,9.39,7.73
uniprot:Q9Y6Z2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.21,0.00,0.00,0.00,1.67,3.23,0.00,0.00,0.00
uniprot:Q9Y6Z5,0.37,0.86,0.00,0.28,9.98,0.00,0.00,0.55,3.62,0.00,⋯,0.00,0.41,0.00,0.00,0.00,0.48,0.20,0.00,0.00,0.81
uniprot:Q9Y6Z7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
library(stringr)
t_sp_dts <- t(small_dts)
tissues <- c(rownames(t_sp_dts))
t_sp_dts <- cbind(t_sp_dts, tissues)
heart_dts <- subset(t_sp_dts, grepl("heart",tissues))
# heart_dts <- 
heart_dts

,uniprot:A0A183,"uniprot:A0A577,uniprot:Q5ZGI7",uniprot:A0A582,uniprot:A0A583,uniprot:A0A597,uniprot:A0A599,uniprot:A0A5C0,"uniprot:A0A962,uniprot:Q96EP0","uniprot:A0AUL9,uniprot:Q2TAZ0","uniprot:A0AUX0,uniprot:E9PLK6,uniprot:Q01432",⋯,uniprot:Q9Y6X9,uniprot:Q9Y6Y0,uniprot:Q9Y6Y8,"uniprot:Q9Y6Y8,uniprot:F5H0L8","uniprot:Q9Y6Y9,uniprot:D0VAW8","uniprot:Q9Y6Y9,uniprot:E5RJJ7",uniprot:Q9Y6Z2,uniprot:Q9Y6Z5,uniprot:Q9Y6Z7,tissues
heart..adult..diseased.post.infarction..donor1.CNhs11757.10050.101G5,0.2,0,0,0.2,0,0,0,0.4,1.6,0.6,⋯,0.4,3.59,0.6,49.32,0.2,3,0,0,0,heart..adult..diseased.post.infarction..donor1.CNhs11757.10050.101G5
heart..adult..diseased..donor1.CNhs11758.10051.101G6,0,0,0,0,0,0,0,0.86,1.29,1.29,⋯,0,2.15,0.43,33.91,0.43,6.87,0,0,0,heart..adult..diseased..donor1.CNhs11758.10051.101G6


In [9]:
rownames(heart_dts) <- c("diseased.infarction", "diseased")
A = heart_dts


In [10]:
B <- A[-(ncol(A))]
rownames(B)<-c(1,0)
head(B)

ERROR: Error in `rownames<-`(`*tmp*`, value = c(1, 0)): attempt to set 'rownames' on an object with no dimensions


In [1]:
rowSums(A)


ERROR: Error in is.data.frame(x): objeto 'A' no encontrado
